# PROJETO TITANIC KAGGLE

### Este notebook é uma adaptação de soluções existentes, realizado para fins de estudos. A solução para este código foi realizada com base no livro [Data Science Solutions](https://www.amazon.com/Data-Science-Solutions-Startup-Workflow/dp/1520545312). 

Descrição

## 1)Importando o projeto

In [2]:
import kaggle

In [3]:
!kaggle competitions download -c titanic

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile


In [5]:
with zipfile.ZipFile("C:/Users/vinic/Documents/GitHub/KG_titanic/titanic.zip", 'r') as zip_ref:
    zip_ref.extractall("C:/Users/vinic/Documents/GitHub/KG_titanic/")

## 2) Contextualização do projeto [finalizar]

ETAPAS QUE DEVEM SER APLICADAS (EM ORDEM OU NÃO)

1. Questionamento e definição de problemas;

2. Treinar e testar o DF;

3. Atribuir as téncicas de Data Wrangle, preparar e testar os dados;

4. Analizar, identificar insights e explorar a base de dados;

5. Definir o modelo preditivo e resolver o problema;

6. Aplicar técnicas de visualização de dados, apresentando os passos e a solução final;

7. Submeter os resultados.


Descrição do projeto ["Titanic Survival Competition".](https://www.kaggle.com/c/titanic)

Descrição dos dados ["Titanic Survival Competition - DATA".](https://www.kaggle.com/c/titanic/data)

>O projeto já apresenta um banco de dados para treino do algorítmo de modelo preditivo. Conhecendo esta base, contendo a lista de passageiros que sobreviveu ou não ao desastre do ticanic, o modelo deve determinar no banco de dados quem sobreviveu ou não.

**Informações**:
- On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. Translated 32% survival rate.
- One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew.
- Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.




## 3) Introdução

### 3.1)Importando Bibliotecas

In [6]:
# bibliotecas de análise e data wrangling
import pandas as pd
import numpy as np
import random as rnd

# bibliotecas de visualização de dados

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# bibliotecas de machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

### 3.2) Aquisição de dados

A biblioteca Pandas foi utilizada para importar os dados e iniciar a tratativa destes.

In [7]:
treino_bd = pd.read_csv(r'C:\Users\vinic\Documents\GitHub\KG_titanic\train.csv')
teste_bd = pd.read_csv(r'C:\Users\vinic\Documents\GitHub\KG_titanic\test.csv')
combine = [treino_bd, teste_bd]

### 3.3) Análise de dados prévia

Essa análise possibilita verificar o nome das colunas. Posteriormente as variáveis (colunas) devem ser avaliadas em variáveis qualitativas ou quantitativas.

 - Variáveis qualitativas nominais (atribuitiva ou binária):
    - PassengerId, Name, Ticket,  Survived, Sex, Cabin Embarked;
 - Variáveis qualitativas ordinais (ordenáveis, geralmente três ou mais categorias):
    - Pclass;
 - Variáveis quantitativas discretas (inteiros):
    - Age, SibSp, Parch.
 - Variáveis quantitativas contínuas (reais):
    - Fare.

In [8]:
treino_bd.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [9]:
treino_bd.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


#### Verificando se existem dados nulos

In [21]:
treino_bd.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [22]:
teste_bd.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [25]:
treino_bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [27]:
## Análise em dados quantitativos
treino_bd.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### Insighs iniciais

 - A amostra contém 891 observações. Representando 40% do número total de passageiros do Titanic (2.224);

 - Survived é uma categoria que varia entre 0 e 1;

 - Aproximadamente 38% das observações na amostra sobreviveram. Dentre os 2.224, 32% sobreviveram;

 - A maioria dos passageiros (>75%) não viajou com parentes ou filhos;



In [31]:
## Análises em dados qualitativos
treino_bd.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Moss, Mr. Albert Johan",male,347082,B96 B98,S
freq,1,577,7,4,644


#### Premissas para análise de dados, considerando 'survival' como a variável alvo:

 - Correlações interessantes:
    1. Maiores de idade e sobrevivência;
    2. Relacionar embarcados com a sobrevivência;
    
 - Correções:
    1. Provavelmente a coluna Ticket apresenta uma quantidade muito elevada de valores duplicados (uma vez que é o valor da passagem). Como não apresenta uma relação com survival, receberá o comando drop.

    2. Cabin apresenta uma quantidade muito grande de valores null. Como não apresenta uma relação com survival, receberá o comando drop.

    3. PassengerId e Nome não contribui em nada para a interpretação ou para a variável survival, receberá o comando drop.

 - Criação:
   1. Criação de uma nova variável com o nome de 'Family', baseado em Parch e SibSp para obter a contagem total de familiares a bordo;
   2. Classificar e ordenar a variável Age e Fare, para análises.

 - Características dos dados:
   1. (Sex = female) onde a maioria sobreviveu;
   2. Crianças (age < ?) onde a maioria sobreviveu;
   3. Passageiros na primeira classe (Pclass = 1) onde a maioria sobreviveu;



#### Para a validação das afirmações feitas, algumas análises adicionais de pivotagem podem ser consideradas





In [44]:
treino_bd[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [50]:
treino_bd[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [52]:
treino_bd[["SibSp","Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by= 'Survived', ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [53]:
treino_bd[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


Logo, é possível afirmar que:
 - Pclass: existe uma relação significativa (>0.5) para a Pclass = 1;
 
 - Sex: Existe uma correlação muito forte para Sex = female, com 74%; 

 - SibSp e Parch: estes campos apresentam correlação zero para alguns itens, sendo assim, faz mais sentido trabalhar com esses dados, para os valores que existem correlação.

### 3.4) Análise visual de dados

